In [1]:
import numpy as np
import pandas as pd

In [2]:
import torch
import torch.optim as optim
# For reproducibility
torch.manual_seed(1)

In [6]:
# df = pd.read_csv("/kaggle/input/2021-ai-w3-p1/train.csv")
train_df = pd.read_csv("./배추가격예측_data/train.csv")
test_df = pd.read_csv("./배추가격예측_data/test.csv")
submit_df = pd.read_csv("./배추가격예측_data/submit_sample.csv")
print(train_df.head())
print(test_df.head())
print(submit_df.head())

       year  avgTemp  minTemp  maxTemp  rainFall  avgPrice
0  20100101     -4.9    -11.0      0.9       0.0      2123
1  20100102     -3.1     -5.5      5.5       0.8      2123
2  20100103     -2.9     -6.9      1.4       0.0      2123
3  20100104     -1.8     -5.1      2.2       5.9      2020
4  20100105     -5.2     -8.7     -1.8       0.7      2060
       year  avgTemp  minTemp  maxTemp  rainFall
0  20170107      4.0     -1.4      9.5       0.1
1  20170120     -1.7     -5.4      1.4       3.3
2  20170223      1.4     -2.0      5.0       0.0
3  20170227      3.6     -2.5     11.1       0.1
4  20170309      5.0     -1.6     11.3       0.0
   Id  Expected
0   0      1000
1   1      1000
2   2      1000
3   3      1000
4   4      1000


In [7]:
x1_train = torch.FloatTensor(train_df['avgTemp'])
x2_train = torch.FloatTensor(train_df['minTemp'])
x3_train = torch.FloatTensor(train_df['maxTemp'])
x4_train = torch.FloatTensor(train_df['rainFall'])
y_train = torch.FloatTensor(train_df['avgPrice'])

In [13]:
# 모델 초기화
w1 = torch.zeros(1, requires_grad=True)
w2 = torch.zeros(1, requires_grad=True)
w3 = torch.zeros(1, requires_grad=True)
w4 = torch.zeros(1,requires_grad= True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([w1, w2, w3,w4, b], lr=1e-3)

nb_epochs = 100000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    hypothesis = x1_train * w1 + x2_train * w2 + x3_train * w3 + x4_train * w4 + b

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 1000 == 0:
        print('Epoch {:4d}/{} w1: {:.3f} w2: {:.3f} w3: {:.3f} w4:{:.3f} b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, w1.item(), w2.item(), w3.item(), w4.item(), b.item(), cost.item()
        ))

Epoch    0/100000 w1: 86.655 w2: 56.931 w3: 121.048 w4:23.573 b: 6.388 Cost: 12013341.000000
Epoch 1000/100000 w1: -69.989 w2: -152.356 w3: 260.954 w4:35.256 b: 325.396 Cost: 2090330.750000
Epoch 2000/100000 w1: -127.657 w2: -104.157 w3: 267.516 w4:31.881 b: 579.812 Cost: 2019757.250000
Epoch 3000/100000 w1: -168.523 w2: -66.673 w3: 268.473 w4:28.889 b: 804.806 Cost: 1965948.625000
Epoch 4000/100000 w1: -196.428 w2: -37.676 w3: 265.352 w4:26.233 b: 1004.134 Cost: 1924508.500000
Epoch 5000/100000 w1: -214.402 w2: -15.388 w3: 259.335 w4:23.872 b: 1181.013 Cost: 1892308.500000
Epoch 6000/100000 w1: -224.834 w2: 1.608 w3: 251.339 w4:21.770 b: 1338.207 Cost: 1867094.250000
Epoch 7000/100000 w1: -229.602 w2: 14.441 w3: 242.062 w4:19.896 b: 1478.099 Cost: 1847218.500000
Epoch 8000/100000 w1: -230.172 w2: 24.010 w3: 232.037 w4:18.224 b: 1602.752 Cost: 1831463.625000
Epoch 9000/100000 w1: -227.692 w2: 31.031 w3: 221.665 w4:16.730 b: 1713.953 Cost: 1818916.375000
Epoch 10000/100000 w1: -223.050 

In [22]:
x1_test = torch.FloatTensor(test_df['avgTemp'])
x2_test = torch.FloatTensor(test_df['minTemp'])
x3_test = torch.FloatTensor(test_df['maxTemp'])
x4_test = torch.FloatTensor(test_df['rainFall'])
y_test = x1_test * w1 + x2_test * w2 + x3_test * w3 + x4_test * w4 + b
print(y_test)

tensor([2997.8792, 2777.1379, 2868.9387, 3113.0391, 3039.6218, 3086.3660,
        3158.4119, 3401.1606, 3380.2937, 3376.5757, 3184.1753, 3468.1128,
        3395.0828, 3414.2251, 3356.1589, 3433.5503, 3389.9026, 3417.3738,
        3392.5610, 3246.0986, 3007.7317, 3022.8804, 2897.9734, 2838.7231],
       grad_fn=<AddBackward0>)


In [29]:
submit_df['Expected'] = y_test.detach().numpy()
submit_df.head()

,Id,Expected
0,0,2997.879150
1,1,2777.137939
2,2,2868.938721
3,3,3113.039062
4,4,3039.621826


In [30]:
submit_df.to_csv("./배추가격예측_data/submit_sample.csv",index=False)